# PYTHON 資料科學期末專題

# [提示]

1. 用Pandas讀入時有時需要注意encoding參數
   
   1.1利用dataframe 去 Creating new feature "Active_case"; Active_case = Confirmed - Deaths – Recovered
   
   1.2 需要注意異常值與缺失值的處理, 注意資料區間, 評估值區間差異過大的問題
   
   1.3 資料的分類, 以期可以分別繪製比對圖形; 繪製出交互作用的地圖

# [基本目標]

1. 繪製出全球的感染與康復人數, 以國家別區分 

2. 利用地理資訊圖表繪製出全球的感染與康復人數


# 參考範例

同學可以參照範例執行, 有問題隨時提問



# <font color='red'> Table of contents </font>
1. <a href='#word'>**Covid-19 - In the world**</a>

1. <a href='#chi'>**Coronavirus in China**</a>

1. <a href='#us'>**Coronavirus in US**</a>


In [1]:
!pip install plotly

In [1]:
# 載入所需要的套件

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
pd.set_option('display.max_rows', None)
from plotly.subplots import make_subplots
import seaborn as sns
import datetime

# 進行資料讀入, 檢視與前處理

In [2]:
# 利用 PANDAS 讀入DATA

data = pd.read_csv('covid_19_data.csv')

In [3]:
# 檢視資料集
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156292 entries, 0 to 156291
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   SNo              156292 non-null  int64  
 1   ObservationDate  156292 non-null  object 
 2   Province/State   111979 non-null  object 
 3   Country/Region   156292 non-null  object 
 4   Last Update      156292 non-null  object 
 5   Confirmed        156292 non-null  float64
 6   Deaths           156292 non-null  float64
 7   Recovered        156292 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 9.5+ MB


In [4]:
# 缺失值處理
#Percentage of NAN Values, 計算缺失值在資料內的比例
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN

,column_name,percentage
0,SNo,0.000000
1,ObservationDate,0.000000
2,Province/State,28.352699
3,Country/Region,0.000000
4,Last Update,0.000000
5,Confirmed,0.000000
6,Deaths,0.000000
7,Recovered,0.000000


In [5]:
# 34 % of Province/State are unknown, 所以我們要把 NAN 的值替換成 Unknown.
data["Province/State"]= data["Province/State"].fillna('Unknown')

In [6]:
#查看資料
data.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0


In [7]:
# 資料型別轉換, 利於 ARRAY
# 把下列資料換成 INT 整數型別 : "Confirmed","Deaths" and "Recovered" 
data[["Confirmed","Deaths","Recovered"]] =data[["Confirmed","Deaths","Recovered"]].astype(int)

In [8]:
# 為了識別簡易, 把 "Mainland China" 用 "China" 替代
data['Country/Region'] = data['Country/Region'].replace('Mainland China', 'China')

In [9]:
# 原始資料集並無感染的欄位, 利用資料處理新增一項目 "Active_case"
# Active_case = Confirmed - Deaths - Recovered

data['Active_case'] = data['Confirmed'] - data['Deaths'] - data['Recovered']
data.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Active_case
0,1,01/22/2020,Anhui,China,1/22/2020 17:00,1,0,0,1
1,2,01/22/2020,Beijing,China,1/22/2020 17:00,14,0,0,14
2,3,01/22/2020,Chongqing,China,1/22/2020 17:00,6,0,0,6
3,4,01/22/2020,Fujian,China,1/22/2020 17:00,1,0,0,1
4,5,01/22/2020,Gansu,China,1/22/2020 17:00,0,0,0,0


In [10]:
# 更新INDEX

Data = data[data['ObservationDate'] == max(data['ObservationDate'])].reset_index()
Data.head()

,index,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Active_case
0,155534,155535,11/15/2020,Unknown,Afghanistan,2020-11-16 05:25:57,43240,1617,35092,6531
1,155535,155536,11/15/2020,Unknown,Albania,2020-11-16 05:25:57,27830,623,12889,14318
2,155536,155537,11/15/2020,Unknown,Algeria,2020-11-16 05:25:57,67679,2154,44633,20892
3,155537,155538,11/15/2020,Unknown,Andorra,2020-11-16 05:25:57,5872,76,4747,1049
4,155538,155539,11/15/2020,Unknown,Angola,2020-11-16 05:25:57,13451,322,6444,6685


# 開始使用資料視覺化顯示結果


<a id='word'></a>
#  <font color='red'> **Covid-19 - In the world** </font>

In [11]:
# 整理資料
Data_world = Data.groupby(["ObservationDate"])[["Confirmed","Active_case","Recovered","Deaths"]].sum().reset_index()

In [1]:
labels = ["Last Update","Confirmed","Active cases","Recovered","Deaths"]
fig = go.Figure(data=[go.Table(header=dict(values=labels),
                 cells=dict(values=Data_world.loc[0,["ObservationDate","Confirmed","Active_case","Recovered","Deaths"]]))
                     ])
fig.update_layout(
    title='Coronavirus in the word : ',
)
fig.show()

'labels = ["Last Update","Confirmed","Active cases","Recovered","Deaths"]\nfig = go.Figure(data=[go.Table(header=dict(values=labels),\n                 cells=dict(values=Data_world.loc[0,["ObservationDate","Confirmed","Active_case","Recovered","Deaths"]]))\n                     ])\nfig.update_layout(\n    title=\'Coronavirus in the word : \',\n)\nfig.show()'

In [2]:
# 繪製圓餅圖
labels = ["Active cases","Recovered","Deaths"]
#計算數字
values = Data_world.loc[0, ["Active_case","Recovered","Deaths"]]
#hole=0.7 的空心圓弧
fig = px.pie(Data_world, values=values, names=labels,color_discrete_sequence=['rgb(77,146,33)','rgb(69,144,185)','rgb(77,77,77)']
             ,hole=0.7)
fig.update_layout(
    title='Total cases : '+str(Data_world["Confirmed"][0]),
)
fig.show()

'# 繪製圓餅圖\nlabels = ["Active cases","Recovered","Deaths"]\n#計算數字\nvalues = Data_world.loc[0, ["Active_case","Recovered","Deaths"]]\n#hole=0.7 的空心圓弧\nfig = px.pie(Data_world, values=values, names=labels,color_discrete_sequence=[\'rgb(77,146,33)\',\'rgb(69,144,185)\',\'rgb(77,77,77)\']\n             ,hole=0.7)\nfig.update_layout(\n    title=\'Total cases : \'+str(Data_world["Confirmed"][0]),\n)\nfig.show()'

In [47]:
# Evolution of coronavirus over time
data_over_time= data.groupby(["ObservationDate"])[["Confirmed","Active_case","Recovered","Deaths"]].sum().reset_index()
data_over_time

,ObservationDate,Confirmed,Active_case,Recovered,Deaths
0,01/22/2020,555,510,28,17
1,01/23/2020,653,605,30,18
2,01/24/2020,941,879,36,26
3,01/25/2020,1438,1357,39,42
4,01/26/2020,2118,2010,52,56
5,01/27/2020,2927,2784,61,82
6,01/28/2020,5578,5340,107,131
7,01/29/2020,6165,5906,126,133
8,01/30/2020,8235,7921,143,171
9,01/31/2020,9925,9490,222,213


In [3]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_over_time.index, y=data_over_time['Confirmed'],
                    mode='lines',
                    name='Confirmed cases'))

fig.update_layout(
    title='Evolution of Confirmed cases over time in the word',
        template='plotly_white',
      yaxis_title="Confirmed cases",
    xaxis_title="Days",
)

fig.show()

'fig = go.Figure()\nfig.add_trace(go.Scatter(x=data_over_time.index, y=data_over_time[\'Confirmed\'],\n                    mode=\'lines\',\n                    name=\'Confirmed cases\'))\n\nfig.update_layout(\n    title=\'Evolution of Confirmed cases over time in the word\',\n        template=\'plotly_white\',\n      yaxis_title="Confirmed cases",\n    xaxis_title="Days",\n)\n\nfig.show()'

In [4]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data_over_time.index, y=data_over_time['Active_case'],
                    mode='lines',
                    name='Active cases',line=dict(color="yellow", dash='dot')))

fig.update_layout(
    title='Evolution of Active cases over time in the world',
        template='plotly_dark',
      yaxis_title="Active cases",
    xaxis_title="Days",
)

fig.show()

'fig = go.Figure()\n\nfig.add_trace(go.Scatter(x=data_over_time.index, y=data_over_time[\'Active_case\'],\n                    mode=\'lines\',\n                    name=\'Active cases\',line=dict(color="yellow", dash=\'dot\')))\n\nfig.update_layout(\n    title=\'Evolution of Active cases over time in the world\',\n        template=\'plotly_dark\',\n      yaxis_title="Active cases",\n    xaxis_title="Days",\n)\n\nfig.show()'

In [5]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data_over_time.index, y=data_over_time['Recovered'],
                    mode='lines',
                    name='Recovered cases',marker_color='green'))

fig.update_layout(
    title='Evolution of Recovered cases over time in the world',
        template='plotly_white',
      yaxis_title="Recovered cases",
    xaxis_title="Days",

)

fig.show()

'fig = go.Figure()\n\nfig.add_trace(go.Scatter(x=data_over_time.index, y=data_over_time[\'Recovered\'],\n                    mode=\'lines\',\n                    name=\'Recovered cases\',marker_color=\'green\'))\n\nfig.update_layout(\n    title=\'Evolution of Recovered cases over time in the world\',\n        template=\'plotly_white\',\n      yaxis_title="Recovered cases",\n    xaxis_title="Days",\n\n)\n\nfig.show()'

In [6]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data_over_time.index, y=data_over_time['Deaths'],name='Deaths',
                                   marker_color='black',mode='lines',line=dict( dash='dot') ))

fig.update_layout(
    title='Evolution of Deaths over time in the world',
        template='plotly_white',
     yaxis_title="Deaths",
    xaxis_title="Days",
)

fig.show()

'fig = go.Figure()\n\nfig.add_trace(go.Scatter(x=data_over_time.index, y=data_over_time[\'Deaths\'],name=\'Deaths\',\n                                   marker_color=\'black\',mode=\'lines\',line=dict( dash=\'dot\') ))\n\nfig.update_layout(\n    title=\'Evolution of Deaths over time in the world\',\n        template=\'plotly_white\',\n     yaxis_title="Deaths",\n    xaxis_title="Days",\n)\n\nfig.show()'

In [7]:
fig = go.Figure(go.Bar(
            x=data_over_time['ObservationDate'],
            y=data_over_time['Confirmed'],
           ))
fig.update_layout(
    title='Confirmed Cases In Each Day',
    template='plotly_white',
     xaxis_title="Confirmed Cases",
    yaxis_title="Days",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=data_over_time[\'ObservationDate\'],\n            y=data_over_time[\'Confirmed\'],\n           ))\nfig.update_layout(\n    title=\'Confirmed Cases In Each Day\',\n    template=\'plotly_white\',\n     xaxis_title="Confirmed Cases",\n    yaxis_title="Days",\n)\nfig.show()'

In [8]:
fig = go.Figure(go.Bar(
            x=data_over_time['ObservationDate'],
            y=data_over_time['Active_case'],
    marker_color='rgb(253,187,132)'
           ))
fig.update_layout(
    title='Active Cases In Each Day',
    template='plotly_dark',
     xaxis_title="Active Cases",
    yaxis_title="Days",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=data_over_time[\'ObservationDate\'],\n            y=data_over_time[\'Active_case\'],\n    marker_color=\'rgb(253,187,132)\'\n           ))\nfig.update_layout(\n    title=\'Active Cases In Each Day\',\n    template=\'plotly_dark\',\n     xaxis_title="Active Cases",\n    yaxis_title="Days",\n)\nfig.show()'

In [9]:
fig = go.Figure(go.Bar(
            x=data_over_time['ObservationDate'],
            y=data_over_time['Recovered'],
    marker_color='rgb(178,24,43)'
           ))
fig.update_layout(
    title='Recovered Cases In Each Day',
    template='plotly_white',
     xaxis_title="Recovered Cases",
    yaxis_title="Days",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=data_over_time[\'ObservationDate\'],\n            y=data_over_time[\'Recovered\'],\n    marker_color=\'rgb(178,24,43)\'\n           ))\nfig.update_layout(\n    title=\'Recovered Cases In Each Day\',\n    template=\'plotly_white\',\n     xaxis_title="Recovered Cases",\n    yaxis_title="Days",\n)\nfig.show()'

In [10]:
fig = go.Figure(go.Bar(
            x=data_over_time['ObservationDate'],
            y=data_over_time['Deaths'],
    marker_color='rgb(13,48,100)'
           ))
fig.update_layout(
    title='Deaths In Each Day',
    template='plotly_white',
     xaxis_title="Deaths",
    yaxis_title="Days",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=data_over_time[\'ObservationDate\'],\n            y=data_over_time[\'Deaths\'],\n    marker_color=\'rgb(13,48,100)\'\n           ))\nfig.update_layout(\n    title=\'Deaths In Each Day\',\n    template=\'plotly_white\',\n     xaxis_title="Deaths",\n    yaxis_title="Days",\n)\nfig.show()'

 **Confirmed cases in each Country**

In [79]:
Data_per_country = Data.groupby(["Country/Region"])["Confirmed","Active_case","Recovered","Deaths"].sum().sort_values("Confirmed",ascending=False).reset_index()
Data_per_country

<ipython-input-79-7dca2179fe90>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Country/Region,Confirmed,Active_case,Recovered,Deaths
0,US,11036935,6615837,4174884,246214
1,India,8845127,465478,8249579,130070
2,Brazil,5863093,417843,5279452,165798
3,France,1915713,1733352,139760,42601
4,Russia,1910149,447699,1429565,32885
5,Spain,1458591,1267446,150376,40769
6,UK,1372884,1317737,3121,52026
7,Argentina,1310491,145953,1129102,35436
8,Colombia,1198746,59759,1104956,34031
9,Italy,1178529,712490,420810,45229


In [11]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Country</b>','<b>Confirmed Cases</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
      
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      Data_per_country['Country/Region'],
      Data_per_country['Confirmed'],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])
fig.update_layout(
    title='Confirmed Cases In Each Country',
)
fig.show()

"headerColor = 'grey'\nrowEvenColor = 'lightgrey'\nrowOddColor = 'white'\n\nfig = go.Figure(data=[go.Table(\n  header=dict(\n    values=['<b>Country</b>','<b>Confirmed Cases</b>'],\n    line_color='darkslategray',\n    fill_color=headerColor,\n    align=['left','center'],\n      \n    font=dict(color='white', size=12)\n  ),\n  cells=dict(\n    values=[\n      Data_per_country['Country/Region'],\n      Data_per_country['Confirmed'],\n      ],\n    line_color='darkslategray',\n    # 2-D list of colors for alternating rows\n    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],\n    align = ['left', 'center'],\n    font = dict(color = 'darkslategray', size = 11)\n    ))\n])\nfig.update_layout(\n    title='Confirmed Cases In Each Country',\n)\nfig.show()"

In [12]:
fig = go.Figure(go.Bar(
            x=Data_per_country['Confirmed'],
            y=Data_per_country['Country/Region'],
            orientation='h')) #"h" or "v"
fig.update_layout(
    title='Confirmed Cases In Each Country',
    template='plotly_dark',
     xaxis_title="Confirmed Cases",
    yaxis_title="Countries",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=Data_per_country[\'Confirmed\'],\n            y=Data_per_country[\'Country/Region\'],\n            orientation=\'h\')) #"h" or "v"\nfig.update_layout(\n    title=\'Confirmed Cases In Each Country\',\n    template=\'plotly_dark\',\n     xaxis_title="Confirmed Cases",\n    yaxis_title="Countries",\n)\nfig.show()'

In [13]:
fig = go.Figure(go.Bar(
            x=Data_per_country['Active_case'],
            y=Data_per_country['Country/Region'],
            orientation='h',
            marker_color='#DC3912',))
fig.update_layout(
    title='Active Cases In Each Country',
    template='plotly_white',
    xaxis_title="Active Cases",
    yaxis_title="Countries",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=Data_per_country[\'Active_case\'],\n            y=Data_per_country[\'Country/Region\'],\n            orientation=\'h\',\n            marker_color=\'#DC3912\',))\nfig.update_layout(\n    title=\'Active Cases In Each Country\',\n    template=\'plotly_white\',\n    xaxis_title="Active Cases",\n    yaxis_title="Countries",\n)\nfig.show()'

In [14]:
fig = go.Figure(go.Bar(
            x=Data_per_country['Recovered'],
            y=Data_per_country['Country/Region'],
            orientation='h',
            marker_color='#2CA02C',))
fig.update_layout(
    title='Recovered Cases In Each Country',
    template='plotly_white',
     xaxis_title="Recovered Cases",
    yaxis_title="Countries",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=Data_per_country[\'Recovered\'],\n            y=Data_per_country[\'Country/Region\'],\n            orientation=\'h\',\n            marker_color=\'#2CA02C\',))\nfig.update_layout(\n    title=\'Recovered Cases In Each Country\',\n    template=\'plotly_white\',\n     xaxis_title="Recovered Cases",\n    yaxis_title="Countries",\n)\nfig.show()'

In [15]:
fig = go.Figure(go.Bar(
            x=Data_per_country['Deaths'],
            y=Data_per_country['Country/Region'],
            orientation='h',
            marker_color='black',))
fig.update_layout(
    title='Deaths In Each Country',
    template='plotly_white',
    xaxis_title="Deaths",
    yaxis_title="Countries",
)

fig.show()

'fig = go.Figure(go.Bar(\n            x=Data_per_country[\'Deaths\'],\n            y=Data_per_country[\'Country/Region\'],\n            orientation=\'h\',\n            marker_color=\'black\',))\nfig.update_layout(\n    title=\'Deaths In Each Country\',\n    template=\'plotly_white\',\n    xaxis_title="Deaths",\n    yaxis_title="Countries",\n)\n\nfig.show()'

In [16]:
fig = px.choropleth(Data_per_country, locations=Data_per_country['Country/Region'],
                    color=Data_per_country['Confirmed'],locationmode='country names', 
                    hover_name=Data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_dark', )
fig.update_layout(
    title='Confirmed Cases In Each Country',
)
fig.show()

"fig = px.choropleth(Data_per_country, locations=Data_per_country['Country/Region'],\n                    color=Data_per_country['Confirmed'],locationmode='country names', \n                    hover_name=Data_per_country['Country/Region'], \n                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_dark', )\nfig.update_layout(\n    title='Confirmed Cases In Each Country',\n)\nfig.show()"

In [17]:
fig = px.choropleth(Data_per_country, locations=Data_per_country['Country/Region'],
                    color=Data_per_country['Active_case'],locationmode='country names', 
                    hover_name=Data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_white', )
fig.update_layout(
    title='Active Cases In Each Country',
)
fig.show()

"fig = px.choropleth(Data_per_country, locations=Data_per_country['Country/Region'],\n                    color=Data_per_country['Active_case'],locationmode='country names', \n                    hover_name=Data_per_country['Country/Region'], \n                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_white', )\nfig.update_layout(\n    title='Active Cases In Each Country',\n)\nfig.show()"

In [18]:
fig = px.choropleth(Data_per_country, locations=Data_per_country['Country/Region'],
                    color=Data_per_country['Recovered'],locationmode='country names', 
                    hover_name=Data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_white', )
fig.update_layout(
    title='Recovered Cases In Each Country',
)
fig.show()

"fig = px.choropleth(Data_per_country, locations=Data_per_country['Country/Region'],\n                    color=Data_per_country['Recovered'],locationmode='country names', \n                    hover_name=Data_per_country['Country/Region'], \n                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_white', )\nfig.update_layout(\n    title='Recovered Cases In Each Country',\n)\nfig.show()"

In [19]:
fig = px.choropleth(Data_per_country, locations=Data_per_country['Country/Region'],
                    color=Data_per_country['Deaths'],locationmode='country names', 
                    hover_name=Data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_dark', )
fig.update_layout(
    title='Deaths In Each Country',
)
fig.show()

"fig = px.choropleth(Data_per_country, locations=Data_per_country['Country/Region'],\n                    color=Data_per_country['Deaths'],locationmode='country names', \n                    hover_name=Data_per_country['Country/Region'], \n                    color_continuous_scale=px.colors.sequential.Tealgrn,template='plotly_dark', )\nfig.update_layout(\n    title='Deaths In Each Country',\n)\nfig.show()"

# Evolution of coronavirus over time.

In [100]:
data_per_country = data.groupby(["Country/Region","ObservationDate"])[["Confirmed","Active_case","Recovered","Deaths"]].sum().reset_index().sort_values("ObservationDate",ascending=True).reset_index(drop=True)


In [20]:
fig = px.choropleth(data_per_country, locations=data_per_country['Country/Region'],
                    color=data_per_country['Confirmed'],locationmode='country names', 
                    hover_name=data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.deep,
                    animation_frame="ObservationDate")
fig.update_layout(

    title='Evolution of confirmed cases In Each Country',
)
fig.show()

'fig = px.choropleth(data_per_country, locations=data_per_country[\'Country/Region\'],\n                    color=data_per_country[\'Confirmed\'],locationmode=\'country names\', \n                    hover_name=data_per_country[\'Country/Region\'], \n                    color_continuous_scale=px.colors.sequential.deep,\n                    animation_frame="ObservationDate")\nfig.update_layout(\n\n    title=\'Evolution of confirmed cases In Each Country\',\n)\nfig.show()'

In [21]:
fig = px.choropleth(data_per_country, locations=data_per_country['Country/Region'],
                    color=data_per_country['Active_case'],locationmode='country names', 
                    hover_name=data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,
                    animation_frame="ObservationDate")
fig.update_layout(

    title='Evolution of active cases In Each Country',
    template='plotly_dark'
)
fig.show()

'fig = px.choropleth(data_per_country, locations=data_per_country[\'Country/Region\'],\n                    color=data_per_country[\'Active_case\'],locationmode=\'country names\', \n                    hover_name=data_per_country[\'Country/Region\'], \n                    color_continuous_scale=px.colors.sequential.Tealgrn,\n                    animation_frame="ObservationDate")\nfig.update_layout(\n\n    title=\'Evolution of active cases In Each Country\',\n    template=\'plotly_dark\'\n)\nfig.show()'

In [22]:
fig = px.choropleth(data_per_country, locations=data_per_country['Country/Region'],
                    color=data_per_country['Recovered'],locationmode='country names', 
                    hover_name=data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.deep,
                    animation_frame="ObservationDate")
fig.update_layout(
    title='Evolution of recovered cases In Each Country',
)
fig.show()

'fig = px.choropleth(data_per_country, locations=data_per_country[\'Country/Region\'],\n                    color=data_per_country[\'Recovered\'],locationmode=\'country names\', \n                    hover_name=data_per_country[\'Country/Region\'], \n                    color_continuous_scale=px.colors.sequential.deep,\n                    animation_frame="ObservationDate")\nfig.update_layout(\n    title=\'Evolution of recovered cases In Each Country\',\n)\nfig.show()'

In [23]:
fig = px.choropleth(data_per_country, locations=data_per_country['Country/Region'],
                    color=data_per_country['Deaths'],locationmode='country names', 
                    hover_name=data_per_country['Country/Region'], 
                    color_continuous_scale=px.colors.sequential.Tealgrn,
                    animation_frame="ObservationDate")
fig.update_layout(
    title='Evolution of deaths In Each Country',
    template='plotly_dark'
)
fig.show()

'fig = px.choropleth(data_per_country, locations=data_per_country[\'Country/Region\'],\n                    color=data_per_country[\'Deaths\'],locationmode=\'country names\', \n                    hover_name=data_per_country[\'Country/Region\'], \n                    color_continuous_scale=px.colors.sequential.Tealgrn,\n                    animation_frame="ObservationDate")\nfig.update_layout(\n    title=\'Evolution of deaths In Each Country\',\n    template=\'plotly_dark\'\n)\nfig.show()'

In [24]:
fig = go.Figure(data=[go.Bar(
            x=Data_per_country['Country/Region'][0:10], y=Data_per_country['Confirmed'][0:10],
            text=Data_per_country['Confirmed'][0:10],
            textposition='auto', #"inside", "outside", "auto", "none"
            marker_color='black',
        )])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
        template='plotly_white'

)
fig.show()

'fig = go.Figure(data=[go.Bar(\n            x=Data_per_country[\'Country/Region\'][0:10], y=Data_per_country[\'Confirmed\'][0:10],\n            text=Data_per_country[\'Confirmed\'][0:10],\n            textposition=\'auto\', #"inside", "outside", "auto", "none"\n            marker_color=\'black\',\n        )])\nfig.update_layout(\n    title=\'Most 10 infected Countries\',\n    xaxis_title="Countries",\n    yaxis_title="Confirmed Cases",\n        template=\'plotly_white\'\n\n)\nfig.show()'

In [25]:
fig = go.Figure(data=[go.Scatter(
    x=Data_per_country['Country/Region'][0:10],
    y=Data_per_country['Confirmed'][0:10],
    mode='markers',
    
    marker=dict(
        color=100+np.random.randn(500),
        size=(Data_per_country['Confirmed'][0:10]/25000), #炫炮的大小圓，因為圓大小差異很大，導致比例尺右偏移
        showscale=True
        )
)])

fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    template='plotly_dark'
)
fig.show()

'fig = go.Figure(data=[go.Scatter(\n    x=Data_per_country[\'Country/Region\'][0:10],\n    y=Data_per_country[\'Confirmed\'][0:10],\n    mode=\'markers\',\n    \n    marker=dict(\n        color=100+np.random.randn(500),\n        size=(Data_per_country[\'Confirmed\'][0:10]/25000), #炫炮的大小圓，因為圓大小差異很大，導致比例尺右偏移\n        showscale=True\n        )\n)])\n\nfig.update_layout(\n    title=\'Most 10 infected Countries\',\n    xaxis_title="Countries",\n    yaxis_title="Confirmed Cases",\n    template=\'plotly_dark\'\n)\nfig.show()'

* **Lets start with good news :  **
**Recorvered cases in each Country **

In [41]:
# *計算恢復數 * *Recorvered cases in each Country *
Recovered_per_country = Data.groupby(["Country/Region"])["Recovered"].sum().reset_index().sort_values("Recovered",ascending=False).reset_index(drop=True)


In [26]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Country</b>','<b>Recovered Cases</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      Recovered_per_country['Country/Region'],
      Recovered_per_country['Recovered'],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])
fig.update_layout(
    title='Recovered Cases In Each Country',
)
fig.show()

"headerColor = 'grey'\nrowEvenColor = 'lightgrey'\nrowOddColor = 'white'\n\nfig = go.Figure(data=[go.Table(\n  header=dict(\n    values=['<b>Country</b>','<b>Recovered Cases</b>'],\n    line_color='darkslategray',\n    fill_color=headerColor,\n    align=['left','center'],\n    font=dict(color='white', size=12)\n  ),\n  cells=dict(\n    values=[\n      Recovered_per_country['Country/Region'],\n      Recovered_per_country['Recovered'],\n      ],\n    line_color='darkslategray',\n    # 2-D list of colors for alternating rows\n    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],\n    align = ['left', 'center'],\n    font = dict(color = 'darkslategray', size = 11)\n    ))\n])\nfig.update_layout(\n    title='Recovered Cases In Each Country',\n)\nfig.show()"

In [27]:
fig = px.pie(Recovered_per_country, values=Recovered_per_country['Recovered'], names=Recovered_per_country['Country/Region'],
             title='Recovered cases'
            )
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    template='plotly_white'
)
fig.show()

"fig = px.pie(Recovered_per_country, values=Recovered_per_country['Recovered'], names=Recovered_per_country['Country/Region'],\n             title='Recovered cases'\n            )\nfig.update_traces(textposition='inside', textinfo='percent+label')\nfig.update_layout(\n    template='plotly_white'\n)\nfig.show()"

In [28]:
fig = go.Figure(data=[go.Bar(
            x=Recovered_per_country['Country/Region'][0:10], y=Recovered_per_country['Recovered'][0:10],
            text=Recovered_per_country['Recovered'][0:10],
            textposition='auto',
            marker_color='green',
        )])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Recovered Cases",
    template='plotly_white'
)
fig.show()

'fig = go.Figure(data=[go.Bar(\n            x=Recovered_per_country[\'Country/Region\'][0:10], y=Recovered_per_country[\'Recovered\'][0:10],\n            text=Recovered_per_country[\'Recovered\'][0:10],\n            textposition=\'auto\',\n            marker_color=\'green\',\n        )])\nfig.update_layout(\n    title=\'Most 10 infected Countries\',\n    xaxis_title="Countries",\n    yaxis_title="Recovered Cases",\n    template=\'plotly_white\'\n)\nfig.show()'

In [29]:
fig = go.Figure(data=[go.Scatter(
    x=Recovered_per_country['Country/Region'][0:10],
    y=Recovered_per_country['Recovered'][0:10],
    mode='markers',
    marker=dict(
        color=100+np.random.randn(500),
        size=(Data_per_country['Recovered'][0:10]/18000),
        showscale=True
        )
)])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Recovered Cases",
    template='plotly_white'
)
fig.show()

'fig = go.Figure(data=[go.Scatter(\n    x=Recovered_per_country[\'Country/Region\'][0:10],\n    y=Recovered_per_country[\'Recovered\'][0:10],\n    mode=\'markers\',\n    marker=dict(\n        color=100+np.random.randn(500),\n        size=(Data_per_country[\'Recovered\'][0:10]/18000),\n        showscale=True\n        )\n)])\nfig.update_layout(\n    title=\'Most 10 infected Countries\',\n    xaxis_title="Countries",\n    yaxis_title="Recovered Cases",\n    template=\'plotly_white\'\n)\nfig.show()'

**Active cases in each Country **

In [48]:
Active_per_country = Data.groupby(["Country/Region"])["Active_case"].sum().reset_index().sort_values("Active_case",ascending=False).reset_index(drop=True)


In [30]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Country</b>','<b>Active Cases</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      Active_per_country['Country/Region'],
      Active_per_country['Active_case'],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])
fig.update_layout(
    title='Active Cases In Each Country',
)
fig.show()

"headerColor = 'grey'\nrowEvenColor = 'lightgrey'\nrowOddColor = 'white'\n\nfig = go.Figure(data=[go.Table(\n  header=dict(\n    values=['<b>Country</b>','<b>Active Cases</b>'],\n    line_color='darkslategray',\n    fill_color=headerColor,\n    align=['left','center'],\n    font=dict(color='white', size=12)\n  ),\n  cells=dict(\n    values=[\n      Active_per_country['Country/Region'],\n      Active_per_country['Active_case'],\n      ],\n    line_color='darkslategray',\n    # 2-D list of colors for alternating rows\n    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],\n    align = ['left', 'center'],\n    font = dict(color = 'darkslategray', size = 11)\n    ))\n])\nfig.update_layout(\n    title='Active Cases In Each Country',\n)\nfig.show()"

In [31]:
fig = go.Figure(data=[go.Bar(
            x=Active_per_country['Country/Region'][0:10], y=Active_per_country['Active_case'][0:10],
            text=Active_per_country['Active_case'][0:10],
           
        )])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Active Cases",
    template='plotly_white'
)
fig.show()

'fig = go.Figure(data=[go.Bar(\n            x=Active_per_country[\'Country/Region\'][0:10], y=Active_per_country[\'Active_case\'][0:10],\n            text=Active_per_country[\'Active_case\'][0:10],\n           \n        )])\nfig.update_layout(\n    title=\'Most 10 infected Countries\',\n    xaxis_title="Countries",\n    yaxis_title="Active Cases",\n    template=\'plotly_white\'\n)\nfig.show()'

In [32]:
fig = go.Figure(data=[go.Scatter(
    x=Active_per_country['Country/Region'][0:10],
    y=Active_per_country['Active_case'][0:10],
    mode='markers',
    marker=dict(
        color=10+np.random.randn(200),

        size=Active_per_country['Active_case'][0:10]/15000,
        showscale=True
        )
)])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Active Cases",
        template='plotly_white'

)
fig.show()

'fig = go.Figure(data=[go.Scatter(\n    x=Active_per_country[\'Country/Region\'][0:10],\n    y=Active_per_country[\'Active_case\'][0:10],\n    mode=\'markers\',\n    marker=dict(\n        color=10+np.random.randn(200),\n\n        size=Active_per_country[\'Active_case\'][0:10]/15000,\n        showscale=True\n        )\n)])\nfig.update_layout(\n    title=\'Most 10 infected Countries\',\n    xaxis_title="Countries",\n    yaxis_title="Active Cases",\n        template=\'plotly_white\'\n\n)\nfig.show()'

**Deaths cases in each Country **

In [52]:
Deaths_per_country = Data.groupby(["Country/Region"])["Deaths"].sum().reset_index().sort_values("Deaths",ascending=False).reset_index(drop=True)


In [33]:
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Country</b>','<b>Deaths</b>'],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      Deaths_per_country['Country/Region'],
      Deaths_per_country['Deaths'],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])
fig.update_layout(
    title='Deaths In Each Country',
)
fig.show()

"headerColor = 'grey'\nrowEvenColor = 'lightgrey'\nrowOddColor = 'white'\n\nfig = go.Figure(data=[go.Table(\n  header=dict(\n    values=['<b>Country</b>','<b>Deaths</b>'],\n    line_color='darkslategray',\n    fill_color=headerColor,\n    align=['left','center'],\n    font=dict(color='white', size=12)\n  ),\n  cells=dict(\n    values=[\n      Deaths_per_country['Country/Region'],\n      Deaths_per_country['Deaths'],\n      ],\n    line_color='darkslategray',\n    # 2-D list of colors for alternating rows\n    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*len(Data_per_country)],\n    align = ['left', 'center'],\n    font = dict(color = 'darkslategray', size = 11)\n    ))\n])\nfig.update_layout(\n    title='Deaths In Each Country',\n)\nfig.show()"

In [34]:
fig = go.Figure(data=[go.Bar(
            x=Deaths_per_country['Country/Region'][0:10], y=Deaths_per_country['Deaths'][0:10],
            text=Deaths_per_country['Deaths'][0:10],
            textposition='auto',
            marker_color='black'

        )])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Deaths",
        template='plotly_white'

)
fig.show()

'fig = go.Figure(data=[go.Bar(\n            x=Deaths_per_country[\'Country/Region\'][0:10], y=Deaths_per_country[\'Deaths\'][0:10],\n            text=Deaths_per_country[\'Deaths\'][0:10],\n            textposition=\'auto\',\n            marker_color=\'black\'\n\n        )])\nfig.update_layout(\n    title=\'Most 10 infected Countries\',\n    xaxis_title="Countries",\n    yaxis_title="Deaths",\n        template=\'plotly_white\'\n\n)\nfig.show()'

In [35]:
fig = go.Figure(data=[go.Scatter(
    x=Deaths_per_country['Country/Region'][0:10],
    y=Deaths_per_country['Deaths'][0:10],
    mode='markers',
    marker=dict(
        color=[145, 140, 135, 130, 125, 120,115,110,105,100],
        size=Deaths_per_country['Deaths'][0:10]/1000,
        showscale=True
        )
)])
fig.update_layout(
    title='Most 10 infected Countries',
    xaxis_title="Countries",
    yaxis_title="Deaths",
        template='plotly_white'

)
fig.show()

'fig = go.Figure(data=[go.Scatter(\n    x=Deaths_per_country[\'Country/Region\'][0:10],\n    y=Deaths_per_country[\'Deaths\'][0:10],\n    mode=\'markers\',\n    marker=dict(\n        color=[145, 140, 135, 130, 125, 120,115,110,105,100],\n        size=Deaths_per_country[\'Deaths\'][0:10]/1000,\n        showscale=True\n        )\n)])\nfig.update_layout(\n    title=\'Most 10 infected Countries\',\n    xaxis_title="Countries",\n    yaxis_title="Deaths",\n        template=\'plotly_white\'\n\n)\nfig.show()'

<a id='chi'></a>
#  <font color='red'> **Coronavirus in China** </font>
![image.png](attachment:image.png)

In [56]:
Data_China = data [(data['Country/Region'] == 'China') ].reset_index(drop=True)
Data_China.head()

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Active_case
0,1,01/22/2020,Anhui,China,1/22/2020 17:00,1,0,0,1
1,2,01/22/2020,Beijing,China,1/22/2020 17:00,14,0,0,14
2,3,01/22/2020,Chongqing,China,1/22/2020 17:00,6,0,0,6
3,4,01/22/2020,Fujian,China,1/22/2020 17:00,1,0,0,1
4,5,01/22/2020,Gansu,China,1/22/2020 17:00,0,0,0,0


Get last update in china

In [57]:
Data_china_last = Data_China[Data_China['ObservationDate'] == max(Data_China['ObservationDate'])].reset_index()
Data_china_last.head()

,index,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Active_case
0,9236,155727,11/15/2020,Anhui,China,2020-11-16 05:25:57,992,6,985,1
1,9237,155761,11/15/2020,Beijing,China,2020-11-16 05:25:57,947,9,932,6
2,9238,155811,11/15/2020,Chongqing,China,2020-11-16 05:25:57,589,6,582,1
3,9239,155853,11/15/2020,Fujian,China,2020-11-16 05:25:57,461,1,426,34
4,9240,155858,11/15/2020,Gansu,China,2020-11-16 05:25:57,181,2,177,2


Confirmed cases in every Province/State in china

In [58]:
# 取得確診率
Data_china_per_state= Data_china_last.groupby(["Province/State"])["Confirmed","Active_case","Recovered","Deaths"].sum().reset_index().sort_values("Confirmed",ascending=False).reset_index(drop=True)


<ipython-input-58-e2a4f3a3e293>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [36]:
fig = px.pie(Data_china_per_state, values=Data_china_per_state['Confirmed'], names=Data_china_per_state['Province/State'],
             title='Confirmed cases in China',
            hole=.2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

"fig = px.pie(Data_china_per_state, values=Data_china_per_state['Confirmed'], names=Data_china_per_state['Province/State'],\n             title='Confirmed cases in China',\n            hole=.2)\nfig.update_traces(textposition='inside', textinfo='percent+label')\nfig.show()"

In [37]:
fig = go.Figure(go.Bar(
            x=Data_china_per_state['Active_case'],
            y=Data_china_per_state['Province/State'],
            orientation='h',
            marker_color='#DC3912',))
fig.update_layout(
    title='Active Cases In Each Province/State',
    template='plotly_white',
    xaxis_title="Active Cases",
    yaxis_title="Province/State",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=Data_china_per_state[\'Active_case\'],\n            y=Data_china_per_state[\'Province/State\'],\n            orientation=\'h\',\n            marker_color=\'#DC3912\',))\nfig.update_layout(\n    title=\'Active Cases In Each Province/State\',\n    template=\'plotly_white\',\n    xaxis_title="Active Cases",\n    yaxis_title="Province/State",\n)\nfig.show()'

In [38]:
fig = go.Figure(go.Bar(
            x=Data_china_per_state['Recovered'],
            y=Data_china_per_state['Province/State'],
            orientation='h',
            marker_color='green',))
fig.update_layout(
    title='Active Cases In Each Province/State',
    template='plotly_white',
    xaxis_title="Recovered Cases",
    yaxis_title="Province/State",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=Data_china_per_state[\'Recovered\'],\n            y=Data_china_per_state[\'Province/State\'],\n            orientation=\'h\',\n            marker_color=\'green\',))\nfig.update_layout(\n    title=\'Active Cases In Each Province/State\',\n    template=\'plotly_white\',\n    xaxis_title="Recovered Cases",\n    yaxis_title="Province/State",\n)\nfig.show()'

In [39]:
fig = go.Figure(go.Bar(
            x=Data_china_per_state['Deaths'],
            y=Data_china_per_state['Province/State'],
            orientation='h',
            marker_color='black',))
fig.update_layout(
    title='Deaths In Each Province/State',
    template='plotly_white',
    xaxis_title="Deaths",
    yaxis_title="Province/State",
)
fig.show()

'fig = go.Figure(go.Bar(\n            x=Data_china_per_state[\'Deaths\'],\n            y=Data_china_per_state[\'Province/State\'],\n            orientation=\'h\',\n            marker_color=\'black\',))\nfig.update_layout(\n    title=\'Deaths In Each Province/State\',\n    template=\'plotly_white\',\n    xaxis_title="Deaths",\n    yaxis_title="Province/State",\n)\nfig.show()'

In [63]:
Data_china_total= Data_china_last.groupby(["Country/Region"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)


<ipython-input-63-46c97c8cce41>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [40]:
labels = ["Active cases","Recovered","Deaths"]
values = Data_china_total.loc[0, ["Active_case","Recovered","Deaths"]]
df = px.data.tips()
fig = px.pie(Data_china_total, values=values, names=labels, color_discrete_sequence=['green','royalblue','darkblue'], hole=0.5)
fig.update_layout(
    title='Total cases in China : '+str(Data_china_total["Confirmed"][0]),
)
fig.show()

'labels = ["Active cases","Recovered","Deaths"]\nvalues = Data_china_total.loc[0, ["Active_case","Recovered","Deaths"]]\ndf = px.data.tips()\nfig = px.pie(Data_china_total, values=values, names=labels, color_discrete_sequence=[\'green\',\'royalblue\',\'darkblue\'], hole=0.5)\nfig.update_layout(\n    title=\'Total cases in China : \'+str(Data_china_total["Confirmed"][0]),\n)\nfig.show()'

Evolution of cases over time in China

In [65]:
Data_china_op= Data_China.groupby(["ObservationDate","Country/Region"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)


<ipython-input-65-637c00b2563f>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [41]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Data_china_op['ObservationDate'], y=Data_china_op['Confirmed'],
                    mode='lines',
                    name='Confirmed cases'))


fig.add_trace(go.Scatter(x=Data_china_op['ObservationDate'], y=Data_china_op['Active_case'],
                    mode='lines',
                    name='Active cases',line=dict( dash='dot')))
fig.add_trace(go.Scatter(x=Data_china_op['ObservationDate'], y=Data_china_op['Deaths'],name='Deaths',
                                   marker_color='black',mode='lines',line=dict( dash='dot') ))
fig.add_trace(go.Scatter(x=Data_china_op['ObservationDate'], y=Data_china_op['Recovered'],
                    mode='lines',
                    name='Recovered cases',marker_color='green'))

fig.update_layout(
    title='Evolution of cases over time in China',
        template='plotly_white'

)

fig.show()

"fig = go.Figure()\nfig.add_trace(go.Scatter(x=Data_china_op['ObservationDate'], y=Data_china_op['Confirmed'],\n                    mode='lines',\n                    name='Confirmed cases'))\n\n\nfig.add_trace(go.Scatter(x=Data_china_op['ObservationDate'], y=Data_china_op['Active_case'],\n                    mode='lines',\n                    name='Active cases',line=dict( dash='dot')))\nfig.add_trace(go.Scatter(x=Data_china_op['ObservationDate'], y=Data_china_op['Deaths'],name='Deaths',\n                                   marker_color='black',mode='lines',line=dict( dash='dot') ))\nfig.add_trace(go.Scatter(x=Data_china_op['ObservationDate'], y=Data_china_op['Recovered'],\n                    mode='lines',\n                    name='Recovered cases',marker_color='green'))\n\nfig.update_layout(\n    title='Evolution of cases over time in China',\n        template='plotly_white'\n\n)\n\nfig.show()"

<a id='us'></a>
#  <font color='red'> **Coronavirus in United States** </font>
![image.png](attachment:image.png)

In [67]:
Data_US = data [(data['Country/Region'] == 'US') ].reset_index(drop=True)


In [68]:
Data_us_last = Data_US[Data_US['ObservationDate'] == max(Data_US['ObservationDate'])].reset_index()

In [69]:
Data_us_total= Data_us_last.groupby(["Country/Region"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)


<ipython-input-69-1dcd34c1fc1d>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [42]:
labels = ["Active cases","Recovered","Deaths"]
values = Data_us_total.loc[0, ["Active_case","Recovered","Deaths"]]
df = px.data.tips()
fig = px.pie(Data_us_total, values=values, names=labels, color_discrete_sequence=['royalblue','darkblue','green'], hole=0.5)
fig.update_layout(
    title='Total cases in United States : '+str(Data_us_total["Confirmed"][0]),
)
fig.show()

'labels = ["Active cases","Recovered","Deaths"]\nvalues = Data_us_total.loc[0, ["Active_case","Recovered","Deaths"]]\ndf = px.data.tips()\nfig = px.pie(Data_us_total, values=values, names=labels, color_discrete_sequence=[\'royalblue\',\'darkblue\',\'green\'], hole=0.5)\nfig.update_layout(\n    title=\'Total cases in United States : \'+str(Data_us_total["Confirmed"][0]),\n)\nfig.show()'

Cases in every Province/State in US



In [71]:
Data_us_per_state= Data_us_last.groupby(["Province/State"])["Confirmed","Active_case","Deaths"].sum().reset_index().sort_values("Confirmed",ascending=False).reset_index(drop=True)


<ipython-input-71-f4ffa29eb617>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [43]:
fig = px.pie(Data_us_per_state, values=Data_us_per_state['Confirmed'], names=Data_us_per_state['Province/State'],
             title='Confirmed cases in United States',
            hole=.2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

"fig = px.pie(Data_us_per_state, values=Data_us_per_state['Confirmed'], names=Data_us_per_state['Province/State'],\n             title='Confirmed cases in United States',\n            hole=.2)\nfig.update_traces(textposition='inside', textinfo='percent+label')\nfig.show()"

In [44]:
fig = px.pie(Data_us_per_state, values=Data_us_per_state['Active_case'], names=Data_us_per_state['Province/State'],
             title='Active cases in United States',
            hole=.2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

"fig = px.pie(Data_us_per_state, values=Data_us_per_state['Active_case'], names=Data_us_per_state['Province/State'],\n             title='Active cases in United States',\n            hole=.2)\nfig.update_traces(textposition='inside', textinfo='percent+label')\nfig.show()"

In [45]:
fig = px.pie(Data_us_per_state, values=Data_us_per_state['Deaths'], names=Data_us_per_state['Province/State'],
             title='Deaths in United States',
            hole=.2)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

"fig = px.pie(Data_us_per_state, values=Data_us_per_state['Deaths'], names=Data_us_per_state['Province/State'],\n             title='Deaths in United States',\n            hole=.2)\nfig.update_traces(textposition='inside', textinfo='percent+label')\nfig.show()"

In [75]:
Data_US_op= Data_US.groupby(["ObservationDate","Country/Region"])["Confirmed","Deaths","Recovered","Active_case"].sum().reset_index().reset_index(drop=True)


<ipython-input-75-bd2eb0d6c10b>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [46]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Confirmed'],
                    mode='lines',
                    name='Confirmed cases'))


fig.update_layout(
    title='Evolution of Confirmed cases over time in US',
        template='plotly_white'

)

fig.show()

"fig = go.Figure()\nfig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Confirmed'],\n                    mode='lines',\n                    name='Confirmed cases'))\n\n\nfig.update_layout(\n    title='Evolution of Confirmed cases over time in US',\n        template='plotly_white'\n\n)\n\nfig.show()"

In [47]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Active_case'],
                    mode='lines',
                    name='Active cases',line=dict( dash='dot')))

fig.update_layout(
    title='Evolution of Acitive cases over time in US',
        template='plotly_white'

)

fig.show()

"fig = go.Figure()\n\nfig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Active_case'],\n                    mode='lines',\n                    name='Active cases',line=dict( dash='dot')))\n\nfig.update_layout(\n    title='Evolution of Acitive cases over time in US',\n        template='plotly_white'\n\n)\n\nfig.show()"

In [48]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Recovered'],
                    mode='lines',
                    name='Recovered cases',marker_color='green'))

fig.update_layout(
    title='Evolution of Recovered cases over time in US',
        template='plotly_white'

)

fig.show()

"fig = go.Figure()\n\nfig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Recovered'],\n                    mode='lines',\n                    name='Recovered cases',marker_color='green'))\n\nfig.update_layout(\n    title='Evolution of Recovered cases over time in US',\n        template='plotly_white'\n\n)\n\nfig.show()"

In [49]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Deaths'],name='Deaths',
                                   marker_color='white',mode='lines',line=dict( dash='dot') ))

fig.update_layout(
    title='Evolution of Deaths over time in US',
        template='plotly_dark'

)

fig.show()

"fig = go.Figure()\n\nfig.add_trace(go.Scatter(x=Data_US_op.index, y=Data_US_op['Deaths'],name='Deaths',\n                                   marker_color='white',mode='lines',line=dict( dash='dot') ))\n\nfig.update_layout(\n    title='Evolution of Deaths over time in US',\n        template='plotly_dark'\n\n)\n\nfig.show()"